In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [2]:
%cd
from DeepLearning.Project2.data_loading_preparation import load_audio_dataloaders_validation

/home/kacper


In [3]:
def get_generic_classifier(input_size, hidden_size, output_size):
    classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size),
    )
    return classifier
    
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cuda'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs)
            pred = torch.argmax(yhat, dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [5]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [6]:
def run_experiment(experiment_name, train_func, run, train_params=None):
    path = f"experiments_rnn/{experiment_name}_run_{run}_"
    print("Running experiment for ", path[:-1])
    
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(train_params)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": experiment_name,
                "train_params": train_params,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [7]:
training_device = "cuda"
device = "cuda"
max_epochs = 250

def oneLSTM(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=1024)
    criterion_weights = torch.tensor([1.] * 11 + [0.1]).to(device)
    model = LSTMSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


In [8]:
param_grid = {
    "small": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "smalldouble": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "tiny": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 1,
    },
    "tinydouble": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 2,
    },
    "wide": {
        "hidden_size": 80,
        "input_size": 20,
        "num_layers": 1,
    },
    "medium": {
        "hidden_size": 128,
        "input_size": 20,
        "num_layers": 1,
    },
}


experiment_list = [
    (
        f"trainer_{trainer.__name__}_params_{param_name}", 
        trainer,
        str(run),
        param
    )
    for run in range(11, 21) 
    for trainer in [oneLSTM]
    for param_name, param in param_grid.items()
]

In [9]:
len(experiment_list)

60

In [ ]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        "Experiment:", experiment[0]
    )
    try:
        run_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-04-22T19:55:42.451293 Experiment: trainer_oneLSTM_params_small
Running experiment for  experiments_rnn/trainer_oneLSTM_params_small_run_11
Epoch: 1, Accuracy on validation set: 0.13806368882239795
Epoch: 2, Accuracy on validation set: 0.18935670014960462
Epoch: 3, Accuracy on validation set: 0.22291087839281898
Epoch: 4, Accuracy on validation set: 0.24043599059628126
Epoch: 5, Accuracy on validation set: 0.25881598632186364
Epoch: 6, Accuracy on validation set: 0.2754862150032058
Epoch: 7, Accuracy on validation set: 0.2808292370164565
Epoch: 8, Accuracy on validation set: 0.29258388544560804
Epoch: 9, Accuracy on validation set: 0.30775806796324
Epoch: 10, Accuracy on validation set: 0.31160504381278054
Epoch: 11, Accuracy on validation set: 0.31865783287027144
Epoch: 12, Accuracy on validation set: 0.32720666809147253
Epoch: 13, Accuracy on validation set: 0.34109852532592433
Epoch: 14, Accuracy on validation set: 0.33810643299850396
Epoch: 15, Accuracy on validation set: